In [1]:
from torch_geometric.datasets import Planetoid

import numpy as np
import random
from random import shuffle
import torch.nn as nn
import torch.nn.functional as F
import math
import torch
from qiskit import *
from QGCN_regression import *

import torch.optim as optim
from torch.nn.parameter import Parameter
from sklearn.metrics import f1_score
import scipy.sparse as sp
import numpy as np
from torch_geometric.datasets import Planetoid
from torch_geometric.utils import to_scipy_sparse_matrix
from torch_geometric.nn import GCNConv
import sys
from givens_roatation import GivensRotations
from torch.optim.lr_scheduler import CosineAnnealingLR
import torchquantum as tq
#import torchquantum.layers
#from tq.layers import Op1QAllLayer,Op2QAllLayer
from measure import Measure
from qlayers import QLayer_block1,QLayer_block2,QLayer_block3,QLayer_block4
import os
import matplotlib.pyplot as plt  
import seaborn as sns  


In [2]:
from SeversonDataset_preprocess import mat_to_npy 
cycle_length = 8

mat_to_npy(mode="rdp",cycle_length=cycle_length) # load mat files and delete some samples with anomalies

skip err cell: batch 1, cell_id 0
skip err cell: batch 1, cell_id 1
skip err cell: batch 1, cell_id 2
skip err cell: batch 1, cell_id 3
skip err cell: batch 1, cell_id 4
skip err cell: batch 1, cell_id 5
(4, 256)
(4, 256)
(1, 4, 512)
(4, 256)
(4, 256)
(1, 4, 512)
(4, 256)
(4, 256)
(1, 4, 512)
(4, 256)
(4, 256)
(1, 4, 512)
(4, 256)
(4, 256)
(1, 4, 512)
(4, 256)
(4, 256)
(1, 4, 512)
(4, 256)
(4, 256)
(1, 4, 512)
(4, 256)
(4, 256)
(1, 4, 512)
(1, 4, 512)
(1, 4, 512)
(1, 4, 512)
(1, 4, 512)
(1, 4, 512)
(1, 4, 512)
(1, 4, 512)
(1, 4, 512)
b1c06 finished
(4, 256)
(4, 256)
(1, 4, 512)
(4, 256)
(4, 256)
(1, 4, 512)
(4, 256)
(4, 256)
(1, 4, 512)
(4, 256)
(4, 256)
(1, 4, 512)
(4, 256)
(4, 256)
(1, 4, 512)
(4, 256)
(4, 256)
(1, 4, 512)
(4, 256)
(4, 256)
(1, 4, 512)
(4, 256)
(4, 256)
(1, 4, 512)
(1, 4, 512)
(1, 4, 512)
(1, 4, 512)
(1, 4, 512)
(1, 4, 512)
(1, 4, 512)
(1, 4, 512)
(1, 4, 512)
b1c07 finished
skip err cell: batch 1, cell_id 8
(4, 256)
(4, 256)
(1, 4, 512)
(4, 256)
(4, 256)
(1, 4, 512)


In [3]:
%matplotlib inline
from SeversonDataset_preprocess import train_val_split
from data_preprocessing import Feature_Selector_Dataset
train_val_split(seed=41,cycle_length=8) 

In [5]:
from data_preprocessing import load_Severson
trn_feature, trn_target = load_Severson(training=True, norm=False, part='discharge',points=256)

In [ ]:
print(trn_feature.shape)
print(trn_target.shape)

(728, 4, 256)
(728, 3)


In [6]:
# Reshape the training data from (4550, 4, 500) to (91, 100000)
# 4550 = 91 batteries * 50 cycles, reshaping cycles and features
num_batteries = 91
cycle = 8
num_features = trn_feature.shape[1]
data_per_cycle = trn_feature.shape[2]
#training_data_reshaped = trn_feature.view(91,cycle,data_per_cycle,num_features)

training_data_reshaped = trn_feature.reshape(num_batteries, -1)  # (91, 100000)

# Extract the last target ([:,2]) and reshape into (91, 1)
# Ensure only one target per battery by taking the first target for each set of 50 cycles
training_target_reshaped = trn_target[::cycle, 0].reshape(num_batteries,-1)  # (91, 1)

In [7]:
print(num_features)
print(data_per_cycle)

4
256


In [8]:
print(training_data_reshaped.shape)
print(training_target_reshaped.shape)

(91, 8192)
(91, 1)


In [9]:
np.save("train", training_data_reshaped)
np.save("target", training_target_reshaped)

In [10]:
point=256
cycles=8

In [11]:
# Load data
training_data = np.load("train.npy")
training_data = torch.from_numpy(training_data)
print(training_data.size())
training_target = np.load("target.npy")  # Binary labels for each battery
training_target = torch.from_numpy(training_target)
training_target = torch.flatten(training_target)

# Create dataset with multiple graphs
dataset = BatteryGraphDataset(training_data, training_target,points=(point,cycles))

# Split dataset into train/test
num_train = int(0.8 * len(dataset))
train_dataset = dataset[:num_train]
test_dataset = dataset[num_train:]

# Set up DataLoaders
batch_size = 1  # Each batch contains 1 battery (2048 nodes)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)



# Example batch
for batch in train_loader:
    print("Batch Node Features Shape:", batch.x.shape)
    print("Batch Labels Shape:", batch.y.shape)
    print("Batch Edge Index Shape:", batch.edge_index.shape)
    break


torch.Size([91, 8192])
points = (256, 8)
type(point) = <class 'int'>
type(cycles_per_battery) = <class 'int'>
torch.Size([8192])
torch.Size([8192])
torch.Size([8192])
torch.Size([8192])
torch.Size([8192])
torch.Size([8192])
torch.Size([8192])
torch.Size([8192])
torch.Size([8192])
torch.Size([8192])
torch.Size([8192])
torch.Size([8192])
torch.Size([8192])
torch.Size([8192])
torch.Size([8192])
torch.Size([8192])
torch.Size([8192])
torch.Size([8192])
torch.Size([8192])
torch.Size([8192])
torch.Size([8192])
torch.Size([8192])
torch.Size([8192])
torch.Size([8192])
torch.Size([8192])
torch.Size([8192])
torch.Size([8192])
torch.Size([8192])
torch.Size([8192])
torch.Size([8192])
torch.Size([8192])
torch.Size([8192])
torch.Size([8192])
torch.Size([8192])
torch.Size([8192])
torch.Size([8192])
torch.Size([8192])
torch.Size([8192])
torch.Size([8192])
torch.Size([8192])
torch.Size([8192])
torch.Size([8192])
torch.Size([8192])
torch.Size([8192])
torch.Size([8192])
torch.Size([8192])
torch.Size([8192

In [12]:
# Example: Checking a batch
for batch in train_loader:
    print("Batch Node Features Shape:", batch.x.shape)
    print("Batch Labels Shape:", batch.y.shape)
    print("Batch Edge Index Shape:", batch.edge_index.shape)
    break  # Check one batch


Batch Node Features Shape: torch.Size([2048, 4])
Batch Labels Shape: torch.Size([2048])
Batch Edge Index Shape: torch.Size([2, 7664])


In [13]:
print(dataset)

BatteryGraphDataset(91)


In [14]:
print(dataset[0])
data = dataset

Data(x=[2048, 4], edge_index=[2, 7664], y=[2048], train_mask=[2048], test_mask=[2048])


In [15]:
data = dataset[0]
n_node =  91
n_train = int(n_node*0.8)
n_feature = 4
n_vqc_wires = int(np.log2(n_feature))
#label0 = 2
#label1 = 3

In [16]:
features = dataset.data_array
labels = dataset.label_array
features = torch.nn.functional.normalize(features, p=2, dim=0)
print('features original shape:',features.shape)

features original shape: torch.Size([91, 8192])


In [17]:
print(labels)

tensor([1637.,  786.,  525.,  560.,  662., 1283.,  520.,  707., 1157.,  482.,
         464.,  666.,  849.,  489.,  623.,  857., 1934., 1266.,  546.,  562.,
         634.,  824.,  326., 1049.,  615., 1001.,  988.,  505.,  795.,  514.,
        1052.,  509., 1155.,  524.,  729.,  934.,  827., 1008.,  614.,  487.,
        1092.,  532.,  852.,  922.,  487.,  480.,  649.,  507., 1114.,  915.,
        1641., 1800., 1154.,  507., 1012.,  858., 1062., 1835.,  981., 1015.,
         812.,  557.,  540.,  492.,  701.,  931.,  597.,  531.,  816.,  702.,
         523.,  855.,  740.,  449.,  785.,  730., 1045.,  827.,  868.,  486.,
         860.,  526.,  361., 1027.,  840.,  755.,  522.,  700., 1077.,  874.,
         519.], dtype=torch.float64)


In [18]:
shuffle_index = np.array(range(labels.shape[0]))
print(shuffle_index)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90]


In [19]:
data = data.subgraph(torch.tensor(shuffle_index[0:n_node]))

In [20]:
train_index = torch.zeros_like(data.train_mask)
train_index[0:n_train] = 1
train_index = train_index.to(torch.bool)
test_index = torch.zeros_like(data.test_mask)
test_index[n_train:n_node] =1
test_index = test_index.to(torch.bool)
print(train_index.sum(),test_index.sum())
print(train_index.shape)
print(test_index.shape)

tensor(72) tensor(19)
torch.Size([91])
torch.Size([91])


In [21]:
nclass = labels.max().item()+1
edges = data.edge_index.T


print('edge.shape:',edges.shape)
pq_series = []

for i in range(edges.shape[0]):
    # if edges[i][0].item() in range(n_train,n_node) and edges[i][1].item() in range(n_train,n_node):
    #     continue
    if edges[i][0].item() < edges[i][1].item():
        pq_series.append([edges[i][0].item(), edges[i][1].item()])


edge_index = torch.tensor(pq_series).T
# data.edge_index = edge_index
for i in range(int(max(labels))+1):
    print( 'label = ',i,'sum =',(labels==i).sum())

print('edge.shape:',edge_index.shape)

# sys.exit(0)
print(nclass)
print(features.shape)
print(labels.shape)

edge.shape: torch.Size([180, 2])
label =  0 sum = tensor(0)
label =  1 sum = tensor(0)
label =  2 sum = tensor(0)
label =  3 sum = tensor(0)
label =  4 sum = tensor(0)
label =  5 sum = tensor(0)
label =  6 sum = tensor(0)
label =  7 sum = tensor(0)
label =  8 sum = tensor(0)
label =  9 sum = tensor(0)
label =  10 sum = tensor(0)
label =  11 sum = tensor(0)
label =  12 sum = tensor(0)
label =  13 sum = tensor(0)
label =  14 sum = tensor(0)
label =  15 sum = tensor(0)
label =  16 sum = tensor(0)
label =  17 sum = tensor(0)
label =  18 sum = tensor(0)
label =  19 sum = tensor(0)
label =  20 sum = tensor(0)
label =  21 sum = tensor(0)
label =  22 sum = tensor(0)
label =  23 sum = tensor(0)
label =  24 sum = tensor(0)
label =  25 sum = tensor(0)
label =  26 sum = tensor(0)
label =  27 sum = tensor(0)
label =  28 sum = tensor(0)
label =  29 sum = tensor(0)
label =  30 sum = tensor(0)
label =  31 sum = tensor(0)
label =  32 sum = tensor(0)
label =  33 sum = tensor(0)
label =  34 sum = tensor(

In [22]:
from torch_geometric.loader import DataLoader

batch_size = 1
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Initialize model with BatteryGraphDataset
#model = QGCN(dataset[0], n_feature=4)  
#train(model, train_loader, epochs=50)

# Evaluate on test set
test_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
#test(model, test_loader)


In [26]:
#model = GCN(nfeat=features.shape[1], nclass=nclass,edge_index= data.edge_index)
model = QGCN(dataset[0], n_feature=4)  


device = 'cpu'
model = model.to(device)
#data = dataset.to(device)
train(model, train_loader, epochs=40)
#edge_index and shuffle index

num: 7664
Epoch 1/40, Loss: nan
Epoch 2/40, Loss: nan
Epoch 3/40, Loss: nan
Epoch 4/40, Loss: nan
Epoch 5/40, Loss: nan
Epoch 6/40, Loss: nan
Epoch 7/40, Loss: nan
Epoch 8/40, Loss: nan
Epoch 9/40, Loss: nan
Epoch 10/40, Loss: nan
Epoch 11/40, Loss: nan
Epoch 12/40, Loss: nan
Epoch 13/40, Loss: nan
Epoch 14/40, Loss: nan
Epoch 15/40, Loss: nan
Epoch 16/40, Loss: nan
Epoch 17/40, Loss: nan
Epoch 18/40, Loss: nan
Epoch 19/40, Loss: nan
Epoch 20/40, Loss: nan
Epoch 21/40, Loss: nan
Epoch 22/40, Loss: nan
Epoch 23/40, Loss: nan
Epoch 24/40, Loss: nan
Epoch 25/40, Loss: nan
Epoch 26/40, Loss: nan
Epoch 27/40, Loss: nan
Epoch 28/40, Loss: nan
Epoch 29/40, Loss: nan
Epoch 30/40, Loss: nan
Epoch 31/40, Loss: nan
Epoch 32/40, Loss: nan
Epoch 33/40, Loss: nan
Epoch 34/40, Loss: nan
Epoch 35/40, Loss: nan
Epoch 36/40, Loss: nan
Epoch 37/40, Loss: nan
Epoch 38/40, Loss: nan
Epoch 39/40, Loss: nan
Epoch 40/40, Loss: nan


In [27]:
#test_loader = DataLoader(dataset, batch_size=1, shuffle=False,follow_batch=['x'])
#preds, output, acc = test(model, test_loader,test_index=test_index)
test(model, test_loader)

0
[]
1
[]
2
[]
3
[]
4
[]
5
[]
6
[]
7
[]
8
[]
9
[]
10
[]
11
[]
12
[]
13
[]
14
[]
15
[]
16
[]
17
[]
18
[]
19
[]
20
[]
21
[]
22
[]
23
[]
24
[]
25
[]
26
[]
27
[]
28
[]
29
[]
30
[]
31
[]
32
[]
33
[]
34
[]
35
[]
36
[]
37
[]
38
[]
39
[]
40
[]
41
[]
42
[]
43
[]
44
[]
45
[]
46
[]
47
[]
48
[]
49
[]
50
[]
51
[]
52
[]
53
[]
54
[]
55
[]
56
[]
57
[]
58
[]
59
[]
60
[]
61
[]
62
[]
63
[]
64
[]
65
[]
66
[]
67
[]
68
[]
69
[]
70
[]
71
[]
72
[[845.45361328125], [845.3927001953125], [845.2396240234375], [844.9094848632812], [844.3173828125], [844.23291015625], [844.1433715820312], [844.4998168945312], [843.9547119140625], [844.484130859375], [843.6845092773438], [844.0472412109375], [844.2069702148438], [843.9291381835938], [842.91015625], [843.731201171875], [844.2572021484375], [843.2893676757812], [843.0786743164062], [841.672119140625], [842.7254028320312], [840.4030151367188], [839.5283813476562], [836.7034301757812], [832.0531005859375], [821.2598266601562], [873.4222412109375], [853.6000366210938], [

0.0

In [28]:
for batch in test_loader:
    print(batch.y.tolist())  # Convert to list and print
    

[1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0, 1637.0,